In [1]:
from pprint import pprint 
from elasticsearch import Elasticsearch

es= Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'tdRTedevS6WZDfd1oEOoeQ',
 'name': 'd3f17afee3cf',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-01-10T10:08:26.972230187Z',
             'build_flavor': 'default',
             'build_hash': 'd4b391d925c31d262eb767b8b2db8f398103f909',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.12.0',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.17.1'}}


In [2]:
es.indices.delete(index='products', ignore_unavailable=True)
es.indices.create(index='products')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'products'})

In [3]:
import json 
from tqdm import tqdm 

document_ids = [] 
dummy_data = json.load(open("data/dummy_data.json"))
for document in tqdm(dummy_data, total=len(dummy_data)):
    response = es.index(index='products', body=document)
    document_ids.append(response['_id'])

100%|██████████| 3/3 [00:00<00:00, 36.78it/s]


In [4]:
# without bulk api 

from pprint import pprint 

response = es.update(
    index='products',
    id=document_ids[0],
    script={
        "source": "ctx._source.title = params.title",
        "params":{
            "title": "new title"
        }
    },
)

pprint(response.body)

{'_id': 'QAWF9JQBIRImumIuuzMO',
 '_index': 'products',
 '_primary_term': 1,
 '_seq_no': 3,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 2,
 'result': 'updated'}


In [5]:
response = es.update(
    index='products',
    id=document_ids[1],
    script={
        "source": "ctx._source.new_field = 'dummy_value'",
    },
)
pprint(response.body)

{'_id': 'QQWF9JQBIRImumIuuzNI',
 '_index': 'products',
 '_primary_term': 1,
 '_seq_no': 4,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 2,
 'result': 'updated'}


In [6]:
response = es.delete(index='products', id=document_ids[2])
pprint(response.body)

{'_id': 'QgWF9JQBIRImumIuuzNX',
 '_index': 'products',
 '_primary_term': 1,
 '_seq_no': 5,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 2,
 'result': 'deleted'}


In [7]:
es.indices.delete(index='products', ignore_unavailable=True)
es.indices.create(index='products')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'products'})

In [8]:
# bulk api 

response = es.bulk(
    operations=[
        #action 1 
        {
            "index": {
                "_index": "products",
                "_id": "1"
            }
        },
        # source 1 
        {
            "title": "Sample Title 1",
            "text": "This is the first sample document text",
            "created_on": "2024_09_22"
        },
        #action 2 
        {
            "index": {
                "_index": "products",
                "_id": "2"
            }
        },
        # source 2 
        {
            "title": "Sample Title 2",
            "text": "Here is another example of document text",
            "created_on": "2024_09_24"
        },
        #action 3 
        {
            "index": {
                "_index": "products",
                "_id": "3"
            }
        },
        # source 3 
        {
            "title": "Sample Title 3",
            "text": "This is the third sample document text",
            "created_on": "2024_09_24"
        },
        # action 4 
        {
            "update": {
                "_id": "1",
                "_index": "products"
            }
        },
        #source 4
        {
            "doc": {
                "title": "New title"
            }
        },
        # action 5 
        {
            "update": {
                "_id": "2",
                "_index": "products"
            }
        },
        # Source 5
        {
            "doc": {
                "new_field": "dummy_value"
            }
        },
        # Action 6
        {
            "delete": {
                "_index": "products",
                "_id": "3"
            } 
        },
    ],
)

pprint(response.body)

# errors indicates no of errors if any 

pprint(response['errors'])

{'errors': False,
 'items': [{'index': {'_id': '1',
                      '_index': 'products',
                      '_primary_term': 1,
                      '_seq_no': 0,
                      '_shards': {'failed': 0, 'successful': 1, 'total': 2},
                      '_version': 1,
                      'result': 'created',
                      'status': 201}},
           {'index': {'_id': '2',
                      '_index': 'products',
                      '_primary_term': 1,
                      '_seq_no': 1,
                      '_shards': {'failed': 0, 'successful': 1, 'total': 2},
                      '_version': 1,
                      'result': 'created',
                      'status': 201}},
           {'index': {'_id': '3',
                      '_index': 'products',
                      '_primary_term': 1,
                      '_seq_no': 2,
                      '_shards': {'failed': 0, 'successful': 1, 'total': 2},
                      '_version': 1,
        

In [11]:
response['errors']
#or 
response.body['errors']

False